In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from bs4 import BeautifulSoup
from bs4.element import Tag

import requests

import re
import tqdm
import pandas as pd
import time

In [2]:
df = pd.read_csv('../../input/us-patent-phrase-to-phrase-matching/train.csv')

anchors = df['anchor'].unique()

quers = []
for anchor in anchors:
    contexts = df[df.anchor == anchor].context.unique()
    for c in contexts:
        quers.append(anchor + '+' + c)

In [3]:
def get_patent_ids(q, driver):
    """Scrapping google patent search 
    q - query = keyword + CPC code
    N - num of patents for query
    returns - tuple of patents (id, name)
    """   
    patents = []
    for n in range(1):
        url = f'https://patents.google.com/?q={q}&num=50&oq={q}&page={n}'
        driver.get(url)
        time.sleep(10)

        txt = []
        elements = driver.find_elements(By.TAG_NAME, 'a')
        for e in elements:
            href = e.get_attribute('href')
            txt.append(e.text)

        patents = []
        code = re.compile(r'[A-Z][A-Z]\d.')

        for i, t in enumerate(txt):
            if re.match(code, t):
                patents.append((txt[i], txt[i-1]))
        
    return patents

In [4]:
def google_description(pid, driver):
    url = f'https://patents.google.com/patent/{pid}/en'
    driver.get(url)
    time.sleep(5)
    elements = driver.find_elements(by=By.CLASS_NAME, value='patent-text')
    lines=[pid]
    for e in elements:
        lines.append(e.text)
    
    return lines

In [5]:
def search_justify(patent_name, driver):
    url = f"https://patents.justia.com/search?q={patent_name.replace(' ', '+')}"
    driver.get(url)
    e = driver.find_elements(By.CLASS_NAME, 'number')
    out = ''
    try:
        out = e[0].text.split()[-1]
    except IndexError:
        print(f'No search result for: {patent_name}')
    return out

In [19]:
def load_justify(pid):
    url = f'https://patents.justia.com/patent/{pid}'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")

    div_desc = soup.find('div', {'id':'description'})
    
    patent_description = []
    if div_desc:
        patent_description.append(div_desc.text)

    return patent_description

In [20]:
driver = webdriver.Firefox()
driver.implicitly_wait(3)

i = 366

while True:
    q = quers[i]
    print(f'---------Start scrapping {q}, i = {i}------------')
    pids = get_patent_ids(q, driver)
    with open(f'pids/{q}.txt', 'w') as f:
        f.writelines([p[0]+'\n' for p in pids])
        
    try:
        txt = []
        for pid, pname in tqdm.tqdm(pids):
            pnum = search_justify(pname, driver)
            desc = load_justify(pnum)
            txt+=desc
    except WebDriverException:
        print('WebDriverException')
        continue
        
    with open(f'texts/{q}.txt', 'w') as f:
        f.writelines(txt)
    
    i += 1

---------Start scrapping curing resin composition+C08, i = 365------------


  4%|██▋                                                            | 2/46 [00:05<01:57,  2.68s/it]

No search result for: Thermoplastic polyurethane with very good fire resistance


  7%|████                                                           | 3/46 [00:10<02:40,  3.74s/it]

No search result for: Polypropylenes Having Balanced Strain Hardening, Melt Strength, and Shear …


...

No search result for: Technology development of eco-culture by using paddy field in misgurnus …


 32%|███████████████████▊                                          | 16/50 [00:51<01:41,  2.98s/it]

No search result for: The method of rice field ecology cultivation freshwater shrimp


 36%|██████████████████████▎                                       | 18/50 [00:59<01:44,  3.28s/it]


KeyboardInterrupt: 